# Create a local information system of a Thin-Section

---

Table of Content

1. [Import files](#1-import-files)
2. [Create Raster tiles using Gdal](#2-create-raster-tiles-using-gdal)
3. [Convert SHP to Geojson using GDAL ogr2ogr](#3-convert-shp-to-geojson-using-gdal-ogr2ogr)
4. [Add GeoJson overlay to web-viewer](#4-add-geojson-overlay-to-web-viewer)

---

requirements:

- Python 3
- GDAL

In this section we collect input data and global settings.
We assume that we have a raster file of a Thin-section digitalized in any way and we have defined a polygon feature that overlay minerals or other elements of the thin-section.

The poligon feature layer must be in a GIS-readable format.
We use gadl2tiles with no-crs definition

In [1]:
# verify Python version
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
else :
    print("Python 3 detected. OK.")

# verify GDAL version
try:
    from osgeo import gdal
    print(f"GDAL version: {gdal.__version__}")
except ImportError:
    raise ImportError("GDAL not detected.")

Python 3 detected. OK.
GDAL version: 3.8.5


# 1. Import files

In this section we use `upload_files_widgets` from `lis_function.py` to import data and files usinng widgets.

Next, store the file in a local subfolder with the same name of "project name" gived by the user.

## 1.1 File specification details

1. <u>**Shapefile**</u>: the minimum collection of 4 files [.shp, .dbf, .shx, .cpg] in ESRI standard format. the shapefile will be converted in GeoJSON format by `ogr2ogr` subprocess. The Shapefile must cointain polygons of mineral or other element that overlay the raster image.

**Field table definition used in this LIS**

<!-- 

function PopUpContent(feature) {
    var mineral = feature.get('Mineral');
    var popupContent = '<b> Mineral: ' + getMineralName(mineral) + '</b>';
    // popupContent += '<br> L = ' + feature.get('L');
    // popupContent += '<br> W = ' + feature.get('W');
    popupContent += '<br> Orientation = ' + String(feature.get('O')).substring(0, 5) + '°';
    // popupContent += '<br> AR = ' + feature.get('AR');
    popupContent += '<br> Aspect Ratio = ' + String(feature.get('AsR')).substring(0, 5);
    popupContent += '<br> Area = ' + String(feature.get('A')).substring(0, 5) + ' mm²';
    // popupContent += '<br> E = ' + feature.get('E');
    popupContent += '<br> Roundness = ' + String(feature.get('R')).substring(0, 5);
    // popupContent += '<br> P = ' + feature.get('P');
    // popupContent += '<br> C = ' + feature.get('C');
    // popupContent += '<br> Cp = ' + feature.get('Cp');
    popupContent += '<br> Grain Shape Index = ' + String(feature.get('GSI')).substring(0, 5);
    // popupContent += '<br> GSF = ' + feature.get('GSF');
    // popupContent += '<br> S = ' + feature.get('S');
    // popupContent += '<br> EQPC = ' + feature.get('EQPC');
    // popupContent += '<br> El = ' + feature.get('El');
    return popupContent; -->


2. <u>**Image**</u>: The original file raster used for webview, will be created hundreds of raster-tiles, there are in a format readable by `gdal` and browser, we suggest Jpeg or TIFF format

3. <u>**NO-Coordinate Reference System (NO-CRS)**</u>: check that both Shapefile and Image are correctly overlapped in a NO-CRS system. We only use the internal coordinate of the pixel of the raster, polygon vertex defined in Shapefile are referring to a specific pixel of the Imagefile.


In [2]:
# Input widgets

from lis_functions import upload_files_widgets
Shapefile_selector, Jpgfile_selector, projectname = upload_files_widgets()
# Shapefile_selector, Jpgfile_selector, projectname, EPSGcode = upload_files_widgets()


Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile polygon layer', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Note: ouptut folder will be created in the same directory as the input files')

In [3]:
from lis_functions import save_to_temp_dir

# Save file to a temporary directory

files_path = save_to_temp_dir(Shapefile_selector, Jpgfile_selector, projectname.value)

# Run gdal2tiles


# 2. Create Raster tiles using Gdal

Using a subprocess, we run the function `run_gdal2tiles`, that launch the gdal2tiles procedure in terminal.

By default a simple webviewer is written and stored in openalayer.html file inside the project folder.

NOTE: destination folder is also created by `save_to_temp_dir` function before

In [4]:
from lis_functions import run_gdal2tiles

# Create raster tiles using gdal2tiles
# for customization see function in lis_functions.py

run_gdal2tiles(files_path['Raster_path'], projectname.value)

# some information is stored in tilemapsoruce.xml file generated from gdal2tiles

WARNING! - Directory already exists, Files will be overwrited! Continue? (Y).


Generating Base Tiles:


0...10...20...30...40...50...60...70...80...90...100 - done.
0.

Generating Overview Tiles:


..10...20...30...40...50...60...70...80...90...100 - done.
Tiles stored in: PAL11


# 3. Convert SHP to Geojson using GDAL ogr2ogr

Using `ogr2oge` in a subprocess we convert shp in geoJson format


In [5]:
import os

from lis_functions import convert_shp_to_geojson

# Get the input shapefile path from `save_to_temp_dir` cell
input_shp = files_path['Shp_file_path']

# Create the output GeoJSON file path
output_geojson = projectname.value + '/' + os.path.basename(input_shp) +'.geojson'

# Convert SHP to GeoJSON using the function and an ogr2ogr subprocess
convert_shp_to_geojson(input_shp, output_geojson)

Conversion completed: PAL11/PAL11_min_porphy.shp.geojson


# 4. Add GeoJson overlay to web-viewer

We access the openalyers.html defautl file generated by gdal2tiles and add some simple javascript code to overlay the previously created GeoJSON.

1. We use the file `geojson_template.js`, a JavaScript file prepared with the script code to add to the mapviewer; in the next step we simply change the placeholder to the current GeoJSON file.
2. We create the javascript code and add it as a source script to the `openlayers.html` file generated earlier by the gdal2tiles subprocess

The visualization style for polygon overlay can be edited in javascript using openalyers syntax before this process.

**NOTE**: For a properly map overlay viweing in a browser, we assume a nominal reference system as EPSG:3857, the same as  used by OpenLayers by default. If other SRSs are used, the vector overlay on raster tiles is not correct.

This is a fictitious hypothesis only for the execution of the webviewer in HTML code. Like an hack that exploits the OpenLayer libraries, which are instead designed for maps with well-defined reference systems. It only works if the vector layer and raster layer are already overlaid (according to the numerical values of their respective coordinates), and have been saved without SRS definition.

In [28]:
from lis_functions import create_geojson_js, append_js_to_html

# STEP 1 create geojson.js file 

#geojson_file_path = 'PAL11/PAL11.shp.geojson'
geojson_file_path = output_geojson
template_js_path = 'templates/geojson_template.js'
root_path = projectname.value + '/geojson.js'
#root_path = 'PAL11/geojson.js'


create_geojson_js(geojson_file_path, template_js_path, root_path)

# STEP 2 append the js file to the html file

# RUN
js_file_path = root_path

html_file_path = projectname.value + '/' + 'openlayers.html'

new_htmlfile_path = projectname.value + '/index.html' # output file

append_js_to_html(js_file_path, html_file_path, new_htmlfile_path)



File created: PAL11/geojson.js
File updated: PAL11/index.html


# 5. Add popups to the map

Adding a Javascript template for the feature that implement pop-up function to the web-viewer

#TODO workin on `geojson_template.js`

In [29]:


from bs4 import BeautifulSoup

def add_popup(JStemplate,CSSTemplate,Html_path):
    """
    Function to add HTML strings and append JS and CSS to the HTML created with 
    gdal2tiles and %append_js_to_html function.

    This function copy and reneame the template files (JS,CSS) in the
    folder destination, only html file will be modified.

    Args:
        JStemplate (str): path to the javascript template file,
        CSSTemplate (str): path to the CSS template file,
        Html_path (str): path to the html file to modify
    """
    # STEP 1 add html code

    # read html files:
    with open(Html_path, 'r') as f:
        html_content = f.read()
    
    

    # Open html with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create new HTML content
    new_div = """
    <div id="popup" class="ol-popup">
        <a href="#" id="popup-closer" class="ol-popup-closer"></a>
        <div id="popup-content"></div>
    </div>
    """
    # Find the precise point to insert the new HTML
    before_div = soup.find('div', id='map')
    if before_div:
        before_div.insert_after(BeautifulSoup(new_div, 'html.parser'))
    else:
        print('Error: tag id not found in HTML file')


    # Create a new script
    scrbasename = os.path.basename(JStemplate)
    script_tag = soup.new_tag('script', src = scrbasename)

    # create DOM element
    script_tag.string = ' '
    if soup.body:
        soup.body.append(script_tag)
    else:
        print('Error accessing to html file')

    # Create a new CSS link
    cssbasename = os.path.basename(CSSTemplate)
    css_tag = soup.new_tag('link', rel='stylesheet', href=cssbasename)
    # add CSS link to the head
    if soup.head:
        soup.head.append(css_tag)
    else:
        print('Error accessing to html file during CSS link creation')

    with open(Html_path, 'w') as f:
        f.write(str(soup))
    
    print(f'File updated: {Html_path}')


    # STEP 2 copy JS and CSS file to the root folder of html
    import shutil
    rootfolder = os.path.dirname(Html_path)
    shutil.copy2(JStemplate, rootfolder)
    shutil.copy2(CSSTemplate, rootfolder)

    print(f'File copied to {rootfolder}')

    return

add_popup('templates/geojson_popup.js','templates/geojson_popup.css','PAL11/index.html')

File updated: PAL11/index.html
File copied to PAL11


***other way***

1. add html code

```html
<script src="https://cdn.jsdelivr.net/npm/ol-ext@4.0.23/dist/ol-ext.min.js"></script>
# <link href="https://cdn.jsdelivr.net/npm/ol-ext@4.0.23/dist/ol-ext.min.css" rel="stylesheet">
```

2. create new html like:

Copyright (c) 2017-2018 Jean-Marc VIGLINO, released under CeCILL-B (french BSD like) licence: http://www.cecill.info/

https://github.com/Viglino/ol-ext/blob/master/examples/popup/map.popup.feature.html

---

3. in JS modify like this

```javascript
// Select  interaction
  var select = new ol.interaction.Select({
    hitTolerance: 5,
    multi: true,
    condition: ol.events.condition.singleClick
  });
  map.addInteraction(select);

  // Select control
  var popup = new ol.Overlay.PopupFeature({
    popupClass: 'default anim',
    select: select,
    canFix: true,
    /** /
    template: function(f) {
      return {
        title: function(f) { return f.get('nom')+' ('+f.get('id')+')' },
        attributes: { 
          region: { title: 'Région' }, 
          arrond: 'arrond', 
          cantons: 'cantons', 
          communes: 'communes', 
          pop: 'pop' 
        }
      }
    },
    /**/
    template: {
        title: 
          // 'nom',   // only display the name
          function(f) {
            return f.get('nom')+' ('+f.get('id')+')';
          },
        attributes: // [ 'region', 'arrond', 'cantons', 'communes', 'pop' ]
        {
          'region': { title: 'Région' },
          'arrond': { title: 'Arrondissement' },
          'cantons': { title: 'Cantons' },
          'communes': { title: 'Communes' },
          // with prefix and suffix
          'pop': { 
            title: 'Population',  // attribute's title
            before: '',           // something to add before
            format: ol.Overlay.PopupFeature.localString(),  // format as local string
            after: ' hab.'        // something to add after
          },
          // calculated attribute
          'pop2': {
            title: 'Population (kHab.)',  // attribute's title
            format: function(val, f) { 
              return Math.round(parseInt(f.get('pop'))/100).toLocaleString() + ' kHab.' 
            }
          }
          /* Using localString with a date * /
          'date': { 
            title: 'Date', 
            format: ol.Overlay.PopupFeature.localString(undefined, { weekday: 'long', year: 'numeric', month: 'long', day: 'numeric' }) 
          }
          /**/
        }
    }
  });
  map.addOverlay (popup);

  // Event on attribute click
  popup.on('attribute', console.log)
```